# Fish Navigation in Turbulent Flow

## Install L4CasADi and *Dependencies*

In [1]:
import sys

In [2]:
# @title
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!pip install scikit-build cmake ninja
!pip install git+https://github.com/Tim-Salzmann/l4casadi --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 16.8 MB/s eta 0:00:00
  Cloning https://github.com/Tim-Salzmann/l4casadi to /tmp/pip-req-build-i6mc8eus
  Running command git clone --filter=blob:none --quiet https://github.com/Tim-Salzmann/l4casadi /tmp/pip-req-build-i6mc8eus
  Resolved https://github.com/Tim-Salzmann/l4casadi to commit 246db23decda71dab14cb83bb462cc1c63e9d7ea
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 MB 9.4 MB/s eta 0:00:00
  Created wheel for l4casadi: filename=l4casadi-1.3.0-cp310-cp310-linux_x86_64.whl size=46606 sha256=084ce0586a0653793eb26a2ce1ccf7fd90d54ac97b77df795c44b9a31141841c
  Stored in directory: /tmp/pip-ephem-wheel-cache-3xit0at0/wheels/cd/54/8a/b8796f827085bb3682fe49796c0f4fe19ddeec1a76ab4187d1
Successfully built l4casadi


In [3]:
# @title
!git clone https://github.com/Tim-Salzmann/l4casadi /tmp/l4casadi
sys.path.append('/tmp/l4casadi/examples/fish_turbulent_flow')

Cloning into '/tmp/l4casadi'...
remote: Enumerating objects: 437, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 437 (delta 22), reused 41 (delta 15), pack-reused 380
Receiving objects: 100% (437/437), 27.07 MiB | 20.43 MiB/s, done.
Resolving deltas: 100% (200/200), done.


In [4]:
# @title
from trajectory_generation import trajectory_generator_solver
from utils import plot_velocity_field_particle

## Import

In [5]:
import os
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import casadi as cs
import torch
import numpy as np

In [6]:
import l4casadi as l4c

## Optimization

In [15]:
# @title Set Fish Start Point Position
y_start_pos = 1.5 # @param {type:"slider", min:-1.8, max:1.8, step:0.1}
p_start = np.array([7.75, y_start_pos])
p_goal = np.array([-0.85, -0.4])
u_lim = 1
T = 20
N = 151
dt = T / N

### Load PyTorch Turbulent Flow Model

In [8]:
checkpoint = torch.load(
    "/tmp/l4casadi/examples/fish_turbulent_flow/models/turbolent_flow_model.pt",
    map_location=torch.device('cpu'),
)

# Standardization
model = checkpoint["model"]
meanX = checkpoint["mean"]["x"]
stdX = checkpoint["std"]["x"]
meanY = checkpoint["mean"]["y"]
stdY = checkpoint["std"]["y"]

### Create L4CasADi Model from PyTorch Model

In [ ]:
x = cs.MX.sym("x", 3)
xn = (x - meanX) / stdX

y = l4c.L4CasADi(model, name="turbulent_model")(xn)

y = y * stdY + meanY
fU = cs.Function("fU", [x], [y[0]])
fV = cs.Function("fV", [x], [y[1]])

## Optimization for Energy Efficiency
(This can take 1-2 minutes on Colab CPU)

In [ ]:
# Generate solver
nlp = trajectory_generator_solver(
    fU=fU, fV=fV, dt=dt, N=N, T=T, u_lim=u_lim, GT=False)

# Set Initial Guess and Parameters
params = np.vstack([p_start, np.tile(p_goal[:, None], N).T])
u_init = np.zeros((N - 1, 2))
p_init = np.zeros((N, 2))
p_init[:, :] = p_start
x_init = np.vstack([p_init, u_init])

# Solve NLP
x_init_flat = cs.reshape(x_init, 4 * N - 2, 1)
params_flat = cs.reshape(params, (N + 1) * 2, 1)
sol = nlp["solver"](x0=x_init_flat, p=params_flat, lbg=nlp["lbg"], ubg=nlp["ubg"])

# extract solution
p_sol = np.squeeze(sol["x"])[: N * 2].reshape(2, N).T
u_sol = np.squeeze(sol["x"])[N * 2 :].reshape(2, N - 1).T

## Visualize Result

In [11]:
# Generate velocity field for visualization
neval = 25
Xgrid, Ygrid = np.meshgrid(np.linspace(-1, 8, neval), np.linspace(-2, 2, neval))
U = np.zeros((N, neval, neval))
V = np.zeros((N, neval, neval))
for t in range(0, N):
    for i in range(neval):
        for j in range(neval):
              U[t, i, j] = np.squeeze(fU([t * T / (N - 1), Xgrid[i, j], Ygrid[i, j]]))
              V[t, i, j] = np.squeeze(fV([t * T / (N - 1), Xgrid[i, j], Ygrid[i, j]]))

In [ ]:
%%capture
# Create Animation
fig, ax = plt.subplots(figsize=(10, 5.))
frames = N
anim = FuncAnimation(
    fig,
    lambda frame_num: plot_velocity_field_particle(
        Xgrid,
        Ygrid,
        U[frame_num],
        V[frame_num],
        p_sol[max(0, frame_num - 10): frame_num + 1, 0],
        p_sol[max(0, frame_num - 10): frame_num + 1, 1],
        p_start,
        p_goal,
        round(frame_num / frames * T, 3),
    ),
    frames=frames,
    interval=100,
)

anim.save('anim.gif')

In [ ]:
from IPython.display import Image
Image('./anim.gif')